<a href="https://colab.research.google.com/github/Daniel-Benson-Poe/DS-Unit-2-Linear-Models/blob/master/db_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [6]:
# Reduce instances to only those that include one family dwellings that are
# greater than 100,000 and less than 2,000,000
import numpy as np
import pandas as pd
df = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') &
        (df['SALE_PRICE'] > 100000) &
        (df['SALE_PRICE'] < 2000000)] 
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [7]:
# Split into train and test data using January — March 2019 for train and April 2019 for test
# Start by setting datetime
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
# Create a variable to include our cut off
cut = pd.to_datetime('2019-04-01')
# Create our train set
train = df[df['SALE_DATE'] < cut]
# Create our test set
test = df[df['SALE_DATE'] >= cut]
print(train, '\n', test)

      BOROUGH NEIGHBORHOOD  ... SALE_PRICE  SALE_DATE
44          3        OTHER  ...     550000 2019-01-01
61          4        OTHER  ...     200000 2019-01-01
78          2        OTHER  ...     810000 2019-01-02
108         3        OTHER  ...     125000 2019-01-02
111         3        OTHER  ...     620000 2019-01-02
...       ...          ...  ...        ...        ...
18129       5        OTHER  ...     330000 2019-03-29
18130       5        OTHER  ...     690000 2019-03-29
18132       5        OTHER  ...     610949 2019-03-29
18134       5        OTHER  ...     520000 2019-03-29
18147       4        OTHER  ...     104000 2019-03-30

[2507 rows x 21 columns] 
       BOROUGH NEIGHBORHOOD  ... SALE_PRICE  SALE_DATE
18235       2        OTHER  ...     895000 2019-04-01
18239       2        OTHER  ...     253500 2019-04-01
18244       3        OTHER  ...    1300000 2019-04-01
18280       3        OTHER  ...     789000 2019-04-01
18285       3        OTHER  ...     525000 2019-04-01


In [8]:
# Check dates for the train set
train['SALE_DATE'].unique

<bound method Series.unique of 44      2019-01-01
61      2019-01-01
78      2019-01-02
108     2019-01-02
111     2019-01-02
           ...    
18129   2019-03-29
18130   2019-03-29
18132   2019-03-29
18134   2019-03-29
18147   2019-03-30
Name: SALE_DATE, Length: 2507, dtype: datetime64[ns]>

In [9]:
# Check dates for the test set
test['SALE_DATE'].unique()

array(['2019-04-01T00:00:00.000000000', '2019-04-02T00:00:00.000000000',
       '2019-04-03T00:00:00.000000000', '2019-04-04T00:00:00.000000000',
       '2019-04-05T00:00:00.000000000', '2019-04-08T00:00:00.000000000',
       '2019-04-09T00:00:00.000000000', '2019-04-10T00:00:00.000000000',
       '2019-04-11T00:00:00.000000000', '2019-04-12T00:00:00.000000000',
       '2019-04-14T00:00:00.000000000', '2019-04-15T00:00:00.000000000',
       '2019-04-16T00:00:00.000000000', '2019-04-17T00:00:00.000000000',
       '2019-04-18T00:00:00.000000000', '2019-04-19T00:00:00.000000000',
       '2019-04-22T00:00:00.000000000', '2019-04-23T00:00:00.000000000',
       '2019-04-24T00:00:00.000000000', '2019-04-25T00:00:00.000000000',
       '2019-04-26T00:00:00.000000000', '2019-04-29T00:00:00.000000000',
       '2019-04-30T00:00:00.000000000'], dtype='datetime64[ns]')

In [10]:
# Look at the numeric data
train.select_dtypes(include='number').describe()

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,2507.000000,2507.000000,0.0,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.0,2.507000e+03
mean,6758.303949,75.778221,NaN,10993.398484,0.987635,0.016354,1.003989,1473.744715,1944.766653,1.0,6.215737e+05
std,3975.909029,157.531138,NaN,494.291462,0.110532,0.129966,0.171794,599.217635,27.059337,0.0,2.916072e+05
min,21.000000,1.000000,NaN,10301.000000,0.000000,0.000000,0.000000,0.000000,1890.000000,1.0,1.040000e+05
25%,3837.500000,21.000000,NaN,10314.000000,1.000000,0.000000,1.000000,1144.000000,1925.000000,1.0,4.405000e+05
50%,6022.000000,42.000000,NaN,11234.000000,1.000000,0.000000,1.000000,1368.000000,1940.000000,1.0,5.600000e+05
75%,9888.500000,70.000000,NaN,11413.000000,1.000000,0.000000,1.000000,1683.000000,1960.000000,1.0,7.500000e+05
max,16350.000000,2720.000000,NaN,11697.000000,1.000000,2.000000,3.000000,7875.000000,2018.000000,1.0,1.955000e+06


In [11]:
# Look at non numeric data
train.describe(exclude='number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2507,2507,2507,2507,2507,2507,1,2507,2507,2507
unique,5,6,1,2,13,2497,1,887,11,68
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,130-52 LEFFERTS BOULEVARD,RP.,"4,000",A1,2019-01-31 00:00:00
freq,1204,2382,2507,2476,919,2,1,234,919,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [12]:
# Our best selections for one hot encoding include TAX_CLASS_AT_PRESENT which has 2 unique values
# We could also potentially look at one hot encoding for BOROUGH which has 5 unique values
# Select only the columns we are interested in - remove the high cardinality columns
target = 'SALE_PRICE'
high_cardinality = ['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 
                    'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 
                    'LAND_SQUARE_FEET', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_DATE']
# Set our features to exclude the high cardinality values
features = train.columns.drop([target] + high_cardinality)
features

Index(['TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [0]:
# Create our X and y sets
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [14]:
# Let's look at that
X_train.head()

,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,1,5495,801,NaN,11230.0,1.0,0.0,1.0,1325.0,1930.0,1
61,1,7918,72,NaN,11427.0,1.0,0.0,1.0,2001.0,1940.0,1
78,1,4210,19,NaN,10461.0,1.0,0.0,1.0,2043.0,1925.0,1
108,1,5212,69,NaN,11226.0,1.0,0.0,1.0,2680.0,1899.0,1
111,1,7930,121,NaN,11203.0,1.0,0.0,1.0,1872.0,1940.0,1


In [0]:
# Use one hot encoding to convert TAX_CLASS_AT_PRESENT to two seperate numeric columns
import category_encoders as ce

# Instantiate encoder
encoder = ce.one_hot.OneHotEncoder(use_cat_names=True)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

In [16]:
# Let's see what that looks like now
X_test_enc.head()

,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
18235,1,0,5913,878,NaN,10471.0,1.0,0.0,1.0,2272.0,1930.0,1
18239,1,0,5488,48,NaN,10465.0,1.0,0.0,1.0,720.0,1935.0,1
18244,1,0,5936,31,NaN,11209.0,1.0,0.0,1.0,2210.0,1925.0,1
18280,1,0,7813,24,NaN,11210.0,1.0,0.0,1.0,1520.0,1915.0,1
18285,1,0,8831,160,NaN,11229.0,1.0,0.0,1.0,840.0,1925.0,1


In [17]:
X_train_enc.head()

,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,1,0,5495,801,NaN,11230.0,1.0,0.0,1.0,1325.0,1930.0,1
61,1,0,7918,72,NaN,11427.0,1.0,0.0,1.0,2001.0,1940.0,1
78,1,0,4210,19,NaN,10461.0,1.0,0.0,1.0,2043.0,1925.0,1
108,1,0,5212,69,NaN,11226.0,1.0,0.0,1.0,2680.0,1899.0,1
111,1,0,7930,121,NaN,11203.0,1.0,0.0,1.0,1872.0,1940.0,1


In [18]:
# Get rid of our NaN values
X_train_enc.isnull().sum()

TAX_CLASS_AT_PRESENT_1          0
TAX_CLASS_AT_PRESENT_1D         0
BLOCK                           0
LOT                             0
EASE-MENT                    2507
ZIP_CODE                        0
RESIDENTIAL_UNITS               0
COMMERCIAL_UNITS                0
TOTAL_UNITS                     0
GROSS_SQUARE_FEET               0
YEAR_BUILT                      0
TAX_CLASS_AT_TIME_OF_SALE       0
dtype: int64

In [0]:
# Because EASE-MENT is the only column with NaNs and it appears to span the 
# entire dataframe, we will simply drop the column all together
X_train_enc = X_train_enc.drop('EASE-MENT', axis=1)
X_test_enc = X_test_enc.drop('EASE-MENT', axis=1)

In [20]:
# Use SelectKBest to find the best column
from sklearn.feature_selection import SelectKBest, f_regression

# Instantiate starting with k = 10
selector = SelectKBest(score_func=f_regression, k=10)

# Create Kbest sets
X_train_kbest = selector.fit_transform(X_train_enc, y_train) # always fit_transform for train set
X_test_kbest = selector.transform(X_test_enc) # always transform for test set

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [21]:
# Check the shape of our dataframes
X_train_kbest.shape, X_test_kbest.shape

((2507, 10), (644, 10))

In [22]:
# Perfect! 10 columns were selected. Let's see which features were selected
selected = selector.get_support()
selected

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False])

In [23]:
X_train_enc.columns[selected]

Index(['TAX_CLASS_AT_PRESENT_1', 'TAX_CLASS_AT_PRESENT_1D', 'BLOCK', 'LOT',
       'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT'],
      dtype='object')

In [24]:
# Let's create a loop that prints out all the possible k values so we can 
# decide which one would be best
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

mae_list = []

for k in range(1, X_train_enc.shape[1]+1):
  print(f'{k} features')
  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_kbest = selector.fit_transform(X_train_enc, y_train)
  X_test_kbest = selector.transform(X_test_enc)
  model = LinearRegression()
  model.fit(X_train_kbest, y_train)
  y_pred = model.predict(X_test_kbest)
  mae = mean_absolute_error(y_pred, y_test)
  print(f'MAE on test set: ${mae:.2f}')
  mae_list.append(mae)

1 features
MAE on test set: $183640.59
2 features
MAE on test set: $184003.17
3 features
MAE on test set: $183943.35
4 features
MAE on test set: $167050.43
5 features
MAE on test set: $167907.76
6 features
MAE on test set: $167681.56
7 features
MAE on test set: $167636.32
8 features
MAE on test set: $167713.87
9 features
MAE on test set: $167717.76
10 features
MAE on test set: $167503.01
11 features
MAE on test set: $167503.01


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

In [25]:
# Use SelectKBest to find the best column
from sklearn.feature_selection import SelectKBest, f_regression

# Instantiate with k = 4
selector = SelectKBest(score_func=f_regression, k=4)

# Create Kbest sets
X_train_kbest = selector.fit_transform(X_train_enc, y_train) # always fit_transform for train set
X_test_kbest = selector.transform(X_test_enc) # always transform for test set

# Let's see which features were selected
selected = selector.get_support()
X_train_enc.columns[selected]

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


Index(['BLOCK', 'ZIP_CODE', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET'], dtype='object')

In [26]:
X_train_enc

,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,1,0,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1
61,1,0,7918,72,11427.0,1.0,0.0,1.0,2001.0,1940.0,1
78,1,0,4210,19,10461.0,1.0,0.0,1.0,2043.0,1925.0,1
108,1,0,5212,69,11226.0,1.0,0.0,1.0,2680.0,1899.0,1
111,1,0,7930,121,11203.0,1.0,0.0,1.0,1872.0,1940.0,1
...,...,...,...,...,...,...,...,...,...,...,...
18129,1,0,4081,44,10306.0,1.0,0.0,1.0,921.0,1950.0,1
18130,1,0,2373,201,10314.0,1.0,0.0,1.0,2128.0,1980.0,1
18132,1,0,1132,42,10302.0,1.0,0.0,1.0,1807.0,2018.0,1
18134,1,0,3395,37,10305.0,1.0,0.0,1.0,621.0,1930.0,1


In [27]:
# Let's work on Ridge Regression now using the four features above making sure 
# we are progressively regularizing it 

from IPython.display import display, HTML
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt

for alpha in [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]:

  # Fit ridge regression model
  features =  ['BLOCK', 'ZIP_CODE', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET']
  display(HTML(f'Ridge Regression, with alpha={alpha}'))
  model = Ridge(alpha=alpha, normalize=True)
  model.fit(X_train_enc[features], y_train)

  # Get test MAE
  y_pred = model.predict(X_test_enc[features])
  mae = mean_absolute_error(y_test, y_pred)
  display(HTML(f'Test Mean Absolute Error: ${mae:,.0f}')), print('\n')



In [32]:
from IPython.display import display, HTML
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt

for alpha in [0.001, 0.01, 0.1, 1.0, 1, 100.0, 1000.0]:
    
    # Fit Ridge Regression model
    display(HTML(f'Ridge Regression, with alpha={alpha}'))
    model = Ridge(alpha=alpha, normalize=True)
    model.fit(X_train_enc, y_train)
    y_pred = model.predict(X_test_enc)

    # Get Test MAE
    mae = mean_absolute_error(y_test, y_pred)
    display(HTML(f'Test Mean Absolute Error: ${mae:,.0f}')), print('\n')
    

In [31]:
# Let's use RidgeCV to find what the best alpha is
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

from sklearn.linear_model import RidgeCV
ridge = RidgeCV(alphas=alphas, normalize=True)
ridge.fit(X_test_enc, y_test)
ridge.alpha_

0.01